In [1]:
from scripts.utils import *
import torch
import spacy
from torchnlp.word_to_vector import GloVe as glove
from nltk import word_tokenize
from simple_attention_model import SimpleAttentionModel
import pickle
import torch.nn as nn
import torch.optim as optim


%load_ext autoreload
%autoreload 2

In [2]:
torch.autograd.set_detect_anomaly(True)

In [3]:
data = in_memory_data_loader('datasets/train_10k.json')

In [ ]:
val = np.array([len(data2[i]['text']) for i in range(len(data2)) if data2[i]['label'] == 1]).argsort()[-500:]

In [ ]:
indices_in_val = np.array([len(data2[i]['text']) for i in val]).argsort()[-50:]

In [ ]:
len([data2[val[i]]['text'] for i in indices_in_val][25])

In [ ]:
(np.array([data2[i]['text'][0] for i in val]) == '<table>').all()

In [ ]:
(np.sort(np.array([len(data2[i]['text']) for i in range(len(data2))])) > 100).sum()

In [ ]:
len(data2)

In [ ]:
data = data2

In [ ]:
vocab = glove('6B')

In [ ]:
vocab.vectors

In [ ]:
len(vocab.token_to_index)

In [1]:
html_tags = ['<A>', '</A>', '<B>', '</B>', '<BODY>', '</BODY>', '<BR>', '<DIV>', '</DIV>', '<HEAD>', '</HEAD>', '<H1>', '</H1>', '<H2>', '</H2>', '<H3>', '</H3>', '<H4>', '</H4>', '<H5>','</H5>', '<H6>', '</H6>', '<I>', '</I>', '<img>', '<LI>', '</LI>', '<ol>', '</ol>', '<ul>', '</ul>', '<P>', '</P>', '<table>', '<tr>', '<th>', '</th>', '</tr>','</td>', '</table>']

In [2]:
html_tags = [i.lower() for i in html_tags]

In [3]:
html_tags

['<a>',
 '</a>',
 '<b>',
 '</b>',
 '<body>',
 '</body>',
 '<br>',
 '<div>',
 '</div>',
 '<head>',
 '</head>',
 '<h1>',
 '</h1>',
 '<h2>',
 '</h2>',
 '<h3>',
 '</h3>',
 '<h4>',
 '</h4>',
 '<h5>',
 '</h5>',
 '<h6>',
 '</h6>',
 '<i>',
 '</i>',
 '<img>',
 '<li>',
 '</li>',
 '<ol>',
 '</ol>',
 '<ul>',
 '</ul>',
 '<p>',
 '</p>',
 '<table>',
 '<tr>',
 '<th>',
 '</th>',
 '</tr>',
 '</td>',
 '</table>']

In [ ]:
other_tags = ['@@UNK@@', '@@PAD@@']
all_tags = html_tags + other_tags

In [ ]:
print(all_tags)

In [ ]:
for tag in all_tags: 
    if tag not in vocab.token_to_index:
        vocab.token_to_index[tag] = len(vocab.token_to_index)
        vocab.vectors = torch.cat((vocab.vectors, torch.zeros(1,300)), 0)

In [ ]:
len(vocab.token_to_index)

In [ ]:
embedding_matrix = nn.Embedding.from_pretrained(vocab.vectors)

In [ ]:
embedding_matrix

In [ ]:
[vocab.token_to_index[i] for i in data[0]['question']]

In [ ]:
len(vocab.token_to_index)

In [ ]:
import pickle

In [ ]:
f = open('datasets/glove_token_to_index.pkl', 'wb')

In [ ]:
pickle.dump(vocab.token_to_index, f)

In [ ]:
f.close()

In [ ]:
vocab.vectors

In [ ]:
f = open('datasets/glove_vectors.pkl', 'wb')

In [ ]:
pickle.dump(vocab.vectors, f)

In [ ]:
f.close()

In [ ]:
mini_batch = get_mini_batch(data)

In [ ]:
mini_batch[0].keys()

In [ ]:
questions, texts, l =  get_mini_batch(data)

In [ ]:
l = torch.Tensor(l)

In [ ]:
l

In [ ]:
LSTM_question = torch.nn.LSTM(input_size=300, hidden_size=100, batch_first=True, bidirectional=True)

In [ ]:
questions_sample = torch.ones((32,100,300))

In [ ]:
layers, (hid, cell) = LSTM_question(questions_sample)

In [ ]:
hid.shape

In [ ]:
vocab.vectors.shape

In [ ]:
model = SimpleAttentionModel(300, 100)

In [ ]:
# Only indices here because we want to learn embedding matrix too 

questions = torch.zeros((32, 100)).long()
texts = torch.zeros((32, 100)).long()

In [ ]:
output = model(questions, texts)

In [ ]:
import torch.nn as nn
loss = nn.BCELoss()
# loss(output,l)

In [ ]:
print(output.shape)
print(l.shape)

In [ ]:
loss(output.unsqueeze(1), l.unsqueeze(1)).backward()

In [ ]:
model = SimpleAttentionModel(300, 100, vocab.vectors)

In [4]:
embedding_vector_file = open('datasets/glove_vectors.pkl',"rb")
embedding_vectors = pickle.load(embedding_vector_file)

In [5]:
model = SimpleAttentionModel(300, 100, pretrained_embeddings = embedding_vectors)

In [6]:
number_of_iterations = len(data)//32

In [8]:
# questions, texts, labels = get_mini_batch(data)

In [9]:
tokens_to_id_file = open('datasets/glove_token_to_index.pkl', "rb")
tokens_to_index = pickle.load(tokens_to_id_file)

In [10]:
questions, texts, labels = get_mini_batch(data, tokens_to_index)

In [11]:
labels.shape

torch.Size([32, 1])

In [12]:
out = model(questions, texts)

In [13]:
criterion = nn.BCELoss()

In [14]:
loss = criterion(out.unsqueeze(1), labels)

In [15]:
optimizer = optim.Adam(model.parameters(), lr=0.003)

In [17]:
loss.backward()
nn.utils.clip_grad_norm_(model.parameters(), 0.5)
optimizer.step()
optimizer.zero_grad()
model.zero_grad()

/pytorch/torch/csrc/autograd/python_anomaly_mode.cpp:57: UserWarning: Traceback of forward call that caused the error:
  File "/home/gondi/anaconda3/envs/dlt_project/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/gondi/anaconda3/envs/dlt_project/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/gondi/anaconda3/envs/dlt_project/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/gondi/anaconda3/envs/dlt_project/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/home/gondi/anaconda3/envs/dlt_project/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 563, in start
    self.io_loop.start()
  File "/home/gondi/anaconda3/envs/dlt_project/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "/home/gondi/

RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.